In [215]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from crop_image import *
import imutils
from imutils import contours as imcnts
import skimage.io as io
%matplotlib inline

In [216]:
image = cv2.imread('omr.png')
cv2.imshow('image.jpg',image)
cv2.waitKey(0)
number_of_choices=4
number_of_columns=2

In [217]:
gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray scale image.jpg',gray_scale)
cv2.waitKey(0)

-1

In [218]:
"""gaussian = cv2.GaussianBlur(image,(3,3),0)
cv2.imshow('image',gaussian)
cv2.waitKey(0)"""

"gaussian = cv2.GaussianBlur(image,(3,3),0)\ncv2.imshow('image',gaussian)\ncv2.waitKey(0)"

In [219]:
edged = cv2.Canny(gray_scale,50,200)
cv2.imshow('edged.jpg',edged)
cv2.waitKey(0)

-1

In [220]:
contours,_= cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))
  
cv2.drawContours(image, contours, -1,(0,255,0), 3)
  
cv2.imshow('Contours.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

Number of Contours found = 4


In [221]:
#sorting all contours
if(len(contours)>0):
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

In [222]:
cv2.drawContours(image, contours, 0,(0,255,0), 3)
  
cv2.imshow('Contours3.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

In [223]:
paper_contour=None
for cnt in contours:
    peri = cv2.arcLength(cnt,True)
    print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    cv2.drawContours(image, [approx], 0,(0,0,255), 3)
    print(len(approx))
    coordinates=approx.ravel()
    for i in range(0,len(coordinates),2):
        x=coordinates[i]
        y=coordinates[i+1]
        cv2.putText(image, "vertex"+str(i//2), (x, y),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0)) 
    if(len(approx)==4):
        paper_contour=approx
        break
cv2.imshow('Contours2.jpg', image)
cv2.waitKey(0)
image = cv2.imread('omr.png')

1484.610171675682
4


In [224]:
print(paper_contour.shape)

(4, 1, 2)


In [225]:
paper= four_point_transform(image, paper_contour.reshape(4, 2))  
cv2.imshow('paper', paper)
cv2.waitKey(0)

-1

In [226]:
gray_scale_paper= four_point_transform(gray_scale, paper_contour.reshape(4, 2))  
cv2.imshow('paper', gray_scale_paper)
cv2.waitKey(0)

-1

In [227]:
_,thresholded=cv2.threshold(gray_scale_paper,200,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
cv2.imshow('paper1', thresholded)
cv2.waitKey(0)

-1

In [228]:
# _,thresholded=cv2.threshold(gray_scale_paper,200,255, cv2.THRESH_BINARY_INV)
# cv2.imshow('paper2', thresholded)
# cv2.waitKey(0)

In [229]:
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [230]:
def test_cnts(cnts):
    cv2.drawContours(thresholded, cnts, -1,255, 3)
  
    cv2.imshow('Contours.jpg', thresholded)
    cv2.waitKey(0)

In [231]:
pre_question_cnts=[]
value=(15,50)
for cnt in pap_cnts:
    (x,y,w,h)=cv2.boundingRect(cnt)
    aspect_ratio=w/h
    if(w>=value[0] and w<=value[1] and h>=value[0] and h<=value[1] and aspect_ratio>=0.9 and aspect_ratio<=1.1):
        pre_question_cnts.append(cnt)

In [232]:
print(len(pre_question_cnts),len(pap_cnts))

25 75


In [233]:
question_cnts,_=imcnts.sort_contours(pre_question_cnts,method='top-to-bottom')

In [234]:
Answer_key= {0:1, 1:4, 2:0, 3:2}
correct=0

In [235]:
print(len(question_cnts))

25


In [236]:
#test_cnts(question_cnts)

In [237]:
_,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [238]:
colors=[(255,0,0)
,(0,255,0)
,(0,0,255)
,(0,255,255)
,(255,255,0)
,(0,0,255)
,(0,255,255)
,(255,0,255)
,(192,192,192)
,(128,128,128)
,(128,0,0)
,(128,128,0)
,(0,128,0)
,(128,0,128)
,(0,128,128)
,(0,0,128),
       (255,215,0)]

In [239]:
counter=0
qs=0
for (row,i) in enumerate(np.arange(0,len(question_cnts),number_of_columns*number_of_choices)):

    curr_row_cnts,_=imcnts.sort_contours(question_cnts[i:i+number_of_columns*number_of_choices])
    for k in np.arange(0,len(curr_row_cnts),number_of_choices):
        curr_ques_cnts=curr_row_cnts[k:k+number_of_choices]
        color1=colors[qs%len(colors)]
        cv2.drawContours(paper,curr_ques_cnts,-1,color1, 1)
        bubbled=None
        for (j,c) in enumerate(curr_ques_cnts):
            mask = np.zeros(thresholded.shape, dtype="uint8")
            #negative one at the last arguement to fill the contours
            cv2.drawContours(mask, [c], -1, 255, -1)
            mask= cv2.bitwise_and(thresholded, mask)
            total= cv2.countNonZero(mask)

            if bubbled is None or total >bubbled[0]:
                bubbled= (total, j)
            color= (0, 0, 255)
        print(f"qestion#{qs+1}: {chr(bubbled[1]+ord('A'))}")
        qs+=1
    counter+=1
print(counter)
counter=0
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

qestion#1: B
qestion#2: D
qestion#3: B
qestion#4: B
qestion#5: D
qestion#6: C
qestion#7: A
4


In [240]:
print(correct)
correct=0

0


In [241]:
cv2.imshow('Contours.jpg', thresholded)
cv2.waitKey(0)

-1

In [242]:
for i,cnt in enumerate(question_cnts):
    color1=(255,0,0) if i%2==0 else (0,0,255)
    cv2.drawContours(paper,[cnt],-1,color1, 1)
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

In [243]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()



In [244]:
print(contours)

[array([[[132, 205]],

       [[131, 206]],

       [[131, 207]],

       ...,

       [[135, 205]],

       [[134, 205]],

       [[133, 205]]], dtype=int32), array([[[112, 667]],

       [[112, 668]],

       [[113, 669]],

       [[114, 670]],

       [[114, 671]],

       [[115, 671]],

       [[116, 671]],

       [[116, 670]],

       [[116, 669]],

       [[116, 668]],

       [[115, 669]],

       [[114, 668]],

       [[113, 667]]], dtype=int32), array([[[121, 452]],

       [[121, 453]],

       [[121, 454]],

       [[121, 455]],

       [[121, 456]],

       [[121, 457]],

       [[121, 458]],

       [[121, 459]],

       [[121, 460]],

       [[121, 461]],

       [[121, 462]],

       [[121, 463]],

       [[121, 464]],

       [[121, 465]],

       [[121, 466]],

       [[121, 467]],

       [[121, 468]],

       [[121, 469]],

       [[121, 470]],

       [[121, 471]],

       [[121, 472]],

       [[121, 473]],

       [[121, 474]],

       [[121, 475]],

       [[121

In [245]:
f"adham{chr(bubbled[1]+ord('A'))}"

'adhamA'